In [1]:
#Initial environement of computations

import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests


number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set

# Historical target table
Hist_table=HTable.read(1433, with_units=False)
Hist_table


hours,Entries
IntegerAndSexagesimal,Historical
0 ;,"00 ; 02,23"
0 ;,"00 ; 04,46"
0 ;,"00 ; 07,08"
0 ;,"00 ; 09,31"
0 ;,"00 ; 11,54"
0 ;,"00 ; 14,17"
0 ;,"00 ; 16,40"
0 ;,"00 ; 19,03"
0 ;,"00 ; 21,25"


In [3]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(30):
    Test_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)
    Delta_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)


In [4]:
#  mean motion parameter from the Parisian Alfonsine tables
daily_mean=Historical(Sexagesimal("00;57,07,44,19,38,52,56"),6)
REVO=Historical(Sexagesimal("6,0;0,0"),6)

for i in range(30):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=4):
        Test_recomp[i][1]=daily_mean*((i+1)/24)%REVO
        with set_precision(tmode=TruncatureMode.ROUND, pmode=2):
            Test_recomp[i][1]=1*Test_recomp[i][1]
            Delta_recomp[i][1]=3600*(Hist_table[i][1]-Test_recomp[i][1])

In [5]:
# residuals visualisation and statistical summaries 1
B=Delta_recomp.to_pandas()
(B.mean().mean(), B.std().std())

(-0.06666666666666667, nan)

In [7]:
B.astype(float)

,Entries
hours,
0 ;,0.0
0 ;,0.0
0 ;,0.0
0 ;,0.0
0 ;,0.0
0 ;,0.0
0 ;,0.0
0 ;,0.0
0 ;,0.0


In [8]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Saturn arg hours.csv')
B.to_csv('Saturn arg hours.zip', index=False, compression=compression_opts)